<a href="https://colab.research.google.com/github/wILLIEWILLYWILLIe/hw_1/blob/main/mlhw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML2025 Homework 1 - Retrieval Augmented Generation with Agents

## Environment Setup

First, we will mount your own Google Drive and change the working directory.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from re import M
# Change the working directory to somewhere in your Google Drive.
# You could check the path by right clicking on the folder.
%cd /content
!pwd
!ls
%cd drive/MyDrive/Colab_Notebooks/hw1
!pwd

/content
/content
drive  sample_data
/content/drive/MyDrive/Colab_Notebooks/hw1
/content/drive/MyDrive/Colab_Notebooks/hw1


In this section, we install the necessary python packages and download model weights of the quantized version of LLaMA 3.1 8B. Also, download the dataset. Note that the model weight is around 8GB. If you are using your Google Drive as the working directory, make sure you have enough space for the model.

In [6]:
!python3 -m pip install --no-cache-dir llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
!python3 -m pip install googlesearch-python bs4 charset-normalizer requests-html lxml_html_clean

from pathlib import Path
if not Path('./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf').exists():
    !wget https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf
if not Path('./public.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/public.txt
if not Path('./private.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/private.txt

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 162.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 183.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.2
    Uninstalling websockets-14.2:
      Successfully uninstalled websockets-14.2
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
ERROR: pip's depe

In [ ]:
import torch
if not torch.cuda.is_available():
    raise Exception('You are not using the GPU runtime. Change it first or you will suffer from the super slow inference speed!')
else:
    print('You are good to go!')

You are good to go!


## Prepare the LLM and LLM utility function

By default, we will use the quantized version of LLaMA 3.1 8B. you can get full marks on this homework by using the provided LLM and LLM utility function. You can also try out different LLM models.

In the following code block, we will load the downloaded LLM model weights onto the GPU first.
Then, we implemented the generate_response() function so that you can get the generated response from the LLM model more easily.

You can ignore "llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized" warning.

In [ ]:
from llama_cpp import Llama

# Load the model onto GPU
llama3 = Llama(
    "./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf",
    verbose=False,
    n_gpu_layers=-1,
    n_ctx=16384,    # This argument is how many tokens the model can take. The longer the better, but it will consume more memory. 16384 is a proper value for a GPU with 16GB VRAM.
)

def generate_response(_model: Llama, _messages: str) -> str:
    '''
    This function will inference the model with given messages.
    '''
    _output = _model.create_chat_completion(
        _messages,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        max_tokens=512,    # This argument is how many tokens the model can generate, you can change it and observe the differences.
        temperature=0,      # This argument is the randomness of the model. 0 means no randomness. You will get the same result with the same input every time. You can try to set it to different values.
        repeat_penalty=2.0,
    )["choices"][0]["message"]["content"]
    return _output

llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


## Search Tool

The TA has implemented a search tool for you to search certain keywords using Google Search. You can use this tool to search for the relevant **web pages** for the given question. The search tool can be integrated in the following sections.

In [ ]:
from typing import List
from googlesearch import search as _search
from bs4 import BeautifulSoup
from charset_normalizer import detect
import asyncio
from requests_html import AsyncHTMLSession
import urllib3
urllib3.disable_warnings()

async def worker(s:AsyncHTMLSession, url:str):
    try:
        header_response = await asyncio.wait_for(s.head(url, verify=False), timeout=10)
        if 'text/html' not in header_response.headers.get('Content-Type', ''):
            return None
        r = await asyncio.wait_for(s.get(url, verify=False), timeout=10)
        return r.text
    except:
        return None

async def get_htmls(urls):
    session = AsyncHTMLSession()
    tasks = (worker(session, url) for url in urls)
    return await asyncio.gather(*tasks)

async def search(keyword: str, n_results: int=3) -> List[str]:
    '''
    This function will search the keyword and return the text content in the first n_results web pages.

    Warning: You may suffer from HTTP 429 errors if you search too many times in a period of time. This is unavoidable and you should take your own risk if you want to try search more results at once.
    The rate limit is not explicitly announced by Google, hence there's not much we can do except for changing the IP or wait until Google unban you (we don't know how long the penalty will last either).
    '''
    keyword = keyword[:100]
    # First, search the keyword and get the results. Also, get 2 times more results in case some of them are invalid.
    results = list(_search(keyword, n_results * 2, lang="zh", unique=True))
    # Then, get the HTML from the results. Also, the helper function will filter out the non-HTML urls.
    results = await get_htmls(results)
    # Filter out the None values.
    results = [x for x in results if x is not None]
    # Parse the HTML.
    results = [BeautifulSoup(x, 'html.parser') for x in results]
    # Get the text from the HTML and remove the spaces. Also, filter out the non-utf-8 encoding.
    results = [''.join(x.get_text().split()) for x in results if detect(x.encode()).get('encoding') == 'utf-8']
    # Return the first n results.
    return results[:n_results]

## Test the LLM inference pipeline

In [ ]:
# You can try out different questions here.
test_question='請問誰是 Taylor Swift？'
messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。"},    # System prompt
    {"role": "user", "content": test_question}, # User prompt
]

print(generate_response(llama3, messages))

泰勒絲（Taylor Swift）是一位美國歌手、詞曲作家和音樂製作人。她出生於1989年，來自田納西州。她的音乐风格从乡村摇滚发展到流行搖擺，並且她被誉为当代最成功的女艺人的之一。

泰勒絲早期在鄉郊小鎮演唱會時開始發展音樂事業，她推出了多張專輯，包括《Taylor Swift》、《Fearless》，以及後來更為知名的大熱作如 《1989》（2014年）、_reputation（）和 _Lover （）。她的歌曲經常探討愛情、友誼及自我成長等主題。

泰勒絲獲得了許多獎項，包括13座格萊美奖，並且是史上最快達到百萬銷量的女藝人之一。


## Agents

The TA has implemented the Agent class for you. You can use this class to create agents that can interact with the LLM model. The Agent class has the following attributes and methods:
- Attributes:
    - role_description: The role of the agent. For example, if you want this agent to be a history expert, you can set the role_description to "You are a history expert. You will only answer questions based on what really happened in the past. Do not generate any answer if you don't have reliable sources.".
    - task_description: The task of the agent. For example, if you want this agent to answer questions only in yes/no, you can set the task_description to "Please answer the following question in yes/no. Explanations are not needed."
    - llm: Just an indicator of the LLM model used by the agent.
- Method:
    - inference: This method takes a message as input and returns the generated response from the LLM model. The message will first be formatted into proper input for the LLM model. (This is where you can set some global instructions like "Please speak in a polite manner" or "Please provide a detailed explanation".) The generated response will be returned as the output.

In [ ]:
class LLMAgent():
    def __init__(self, role_description: str, task_description: str, llm:str="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF"):
        self.role_description = role_description   # Role means who this agent should act like. e.g. the history expert, the manager......
        self.task_description = task_description    # Task description instructs what task should this agent solve.
        self.llm = llm  # LLM indicates which LLM backend this agent is using.
    def inference(self, message:str) -> str:
        if self.llm == 'bartowski/Meta-Llama-3.1-8B-Instruct-GGUF': # If using the default one.
            # TODO: Design the system prompt and user prompt here.
            # Format the messsages first.
            messages = [
                # Hint: you may want the agents to speak Traditional Chinese only.
                {"role": "system", "content": f"{self.role_description}使用中文時只會使用繁體中文來回問題。"},
                # Hint: you may want the agents to clearly distinguish the task descriptions and the user messages.
                # A proper seperation text rather than a simple line break is recommended.
                {"role": "user", "content": f"{self.task_description}\n===\n{message}"}
            ]

            return generate_response(llama3, messages)
        else:
            # TODO: If you want to use LLMs other than the given one, please implement the inference part on your own.
            return ""

In [ ]:
role_description = "你是音樂與流行文化的專家，負責提供關於藝人與音樂的準確資訊。"
task_description = "請根據使用者的問題，提供簡潔且準確的回答。"
task_description = ""
agent = LLMAgent(role_description, task_description)

# Test the question
test_question = "請問誰是 Taylor Swift？"
response = agent.inference(test_question)
print(response)

泰勒絲（Taylor Swift）是美國的一位知名女歌手、詞曲作家和音樂製作人。她出生於1989年12月13日，肯塔基州曼徹斯特。她的父親是一個銀行經理，她的母校就讀了本頓高中。

泰勒絲從小便對唱跳有濃厚興趣，並在10歲時開始寫歌曲。在2004年的美國達拉斯音樂節上表演後，獲得路易·丹尼爾（Lou Daniel）和Scott Borchetta的注意，被簽下於大機器娛樂公司。泰勒絲發行了多張專輯，其中包括《Taylor Swift》、《Fearless》，以及她最受歡迎的一個作品——2014年的歌曲「Shake It Off」。

她的音樂風格從鄉村搖滾逐漸轉變為流派，影響力廣泛。泰勒絲的個人生活和藝術創作經常受到媒體關注，她也被譽爺美國最具商業價值的人物之一。她曾獲得13座葛萊美獎，並在2010年成為史上第二快達成百萬銷量專輯女歌手。


TODO: Design the role description and task description for each agent.

In [ ]:
# TODO: Design the role and task description for each agent.

# This agent may help you filter out the irrelevant parts in question descriptions.
question_extraction_agent = LLMAgent(
    role_description="你是問題解析專家，擅長從使用者的輸入中提取核心問題，並確保問題清晰且具體。",
    task_description=(
        "請分析以下輸入，提取核心問題並以簡潔的繁體中文重述。\n"
        "若輸入包含多個問題，請選擇最主要的一個。\n"
        "並保留問題重要條件， 例如地點， 跟誰， 年代 區域。"
    )
)

# This agent may help you extract the keywords in a question so that the search tool can find more accurate results.
keyword_extraction_agent = LLMAgent(
    role_description="你是關鍵詞萃取專家，負責從問題中提取最適合搜尋的關鍵詞，包括專有名詞、核心概念或動作及地點、",
    task_description="請從以下問題中提取關鍵詞，確保其具體且具代表性，用逗號分隔。避免過於泛泛的詞彙。"
)

# This agent is the core component that answers the question.
qa_agent = LLMAgent(
    role_description="你是問答生成專家，負責根據問題和相關資訊提供準確、簡潔且符合事實的回答。",
    task_description=(
        "請根據以下問題和篩選後的相關資訊，提供準確的回答。\n"
        "若 `相關資訊` 提供了明確的數字（如分數、金額、時間），請直接使用該數據來回答問題。\n"
        "若 `相關資訊` 沒有明確答案，請明確標記資訊不足，並建議使用者查閱官方網站或聯絡相關部門。\n"
        "請使用正式且易於理解的繁體中文回答，確保資訊清晰。"
    )
)

context_filter_agent = LLMAgent(
    role_description="你是內容篩選專家，負責從搜尋結果中最相關的資訊，簡化內容以提高問答效率。",
    task_description=(
        "請根據以下問題，從相關資料中提取有相關之資訊，並用繁體中文摘要整理。\n"
        "請確保答案與問題相關，優先保留數字資訊（如分數、金額、時間等）。\n"
        "如果有72數字在內，擷取其數字。"
    )
)

## RAG pipeline

TODO: Implement the RAG pipeline.

Please refer to the homework description slides for hints.

Also, there might be more heuristics (e.g. classifying the questions based on their lengths, determining if the question need a search or not, reconfirm the answer before returning it to the user......) that are not shown in the flow charts. You can use your creativity to come up with a better solution!

- Naive approach (simple baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive.png)

- Naive RAG approach (medium baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive_rag.png)

- RAG with agents (strong baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/rag_agent.png)

In [ ]:
def check_len(myinput : str) -> str:
  max_length = 16384
  if len(myinput) >= max_length :
    available_space = max_length - 4
    myinput = myinput[:available_space] + "..."
  return myinput

async def pipeline(question: str) -> str:
    DO_DEBUG = True
    ########################################################
    # TODO: Implement your pipeline.
    # Currently, it only feeds the question directly to the LLM.
    # You may want to get the final results through multiple inferences.
    # Just a quick reminder, make sure your input length is within the limit of the model context window (16384 tokens),
    # You may want to truncate some excessive texts.
    ########################################################
    if DO_DEBUG : print(f'{" "*5}question: {question}')
    # Step 1: Extract the core question
    core_question = question_extraction_agent.inference(question)
    if DO_DEBUG : print(f'{" "*5}core_question: {core_question}')
    if not core_question:
        return "無法解析問題，請重新輸入。"

    # Step 2: Extract keywords from the core question
    keywords = keyword_extraction_agent.inference(question)
    if DO_DEBUG: print(f'{" "*5}key_words: {keywords}')
    if not keywords:
        return "無法提取關鍵詞，請提供更具體的問題。"

    # Step 3: Retrieve context using the provided search function
    raw_search_results = await search(question, n_results=2)
    raw_search_results2 = await search(keywords, n_results=2)
    raw_search_results = raw_search_results + raw_search_results2
    # if DO_DEBUG : print('   raw_result: ', raw_search_results)
    filter_output_tot = ""
    if raw_search_results :
      for idx, raw_result in enumerate(raw_search_results):
        filter_input  = check_len(f"問題：{core_question}\n相關資料: {' '.join(raw_result)}")
        filtered_context = context_filter_agent.inference(filter_input)
        if not filtered_context:
          filtered_context = "未找到足夠相關資訊，根據已有知識回答。"
          if DO_DEBUG : print(filtered_context)
        else :
          filter_output_tot += filtered_context
          if DO_DEBUG :print(f'{" "*5}{idx}-filter_context\n{filtered_context}')
    if not raw_search_results:
        filter_output_tot = "未找到相關資訊，根據已有知識回答。"

    # Step 5: Generate the final answer
    final_input = f"問題：{core_question}\n相關資訊: {filter_output_tot}"
    final_input = check_len(final_input)
    response = qa_agent.inference(final_input)

    # Step 6: Truncate if exceeding context window (using characters as a proxy for tokens)
    max_length = 16384  # Adjust based on actual token limit
    if len(final_input) + len(response) > max_length:
        available_space = max_length - len(final_input) - 4  # Reserve space for ellipsis
        response = response[:available_space] + "..."

    return response

    # return qa_agent.inference(question)

## Answer the questions using your pipeline!

Since Colab has usage limit, you might encounter the disconnections. The following code will save your answer for each question. If you have mounted your Google Drive as instructed, you can just rerun the whole notebook to continue your process.

In [ ]:
from pathlib import Path

# Fill in your student ID first.
STUDENT_ID = "r12942009"

STUDENT_ID = STUDENT_ID.lower()
with open('./public.txt', 'r') as input_f:
    questions = input_f.readlines()
    questions = [l.strip().split(',')[0] for l in questions]
    for id, question in enumerate(questions, 1):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            continue
        # if id != 4 :
        #   continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(f'{"="*40}')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'w') as output_f:
            print(answer, file=output_f)
            pass
        print(f'{"="*40}')

with open('./private.txt', 'r') as input_f:
    questions = input_f.readlines()
    for id, question in enumerate(questions, 31):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(f'{"="*40}')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'a') as output_f:
            print(answer, file=output_f)
        print(f'{"="*40}')

     question: 校歌為學校（包括小學、中學、大學等）宣告或者規定的代表該校的歌曲。用於體現該校的治學理念、辦學理想等學校文化。「虎山雄風飛揚」是哪間學校的校歌歌詞？
     core_question: 核心問題：「虎山雄風飛揚」是哪間學校的校歌？

重要條件：

* 地點/地區: 不明確
（但根據我知道，可能與某些台北或新竹附近的地方有關）
 
最主要的一個問題
     key_words: 虎山雄風飛揚、校歌
     0-filter_context
根據提供的資訊，我們可以得出以下相關信息：

* 「虎山雄風飛揚」是哪間學校校歌？
答案：無法確定，但可能與某些台北或新竹附近的地方有關。

提取數字：
72 (沒有其他可用的 7 位 數)

根據維基百科的資訊，我們可以得出以下相關信息：

* 中國最早的大學校歌是《警醒 歌》，創作於1896年。
* 日本御茶水女子大學（現為東京女子大学）的學校曲「みがかずば」被認定爲日本第一首的学校乐，创造于1875 年。

因此，我們可以得出以下結論：

答案：無法確定的台北或新竹附近地方的一間學府
     1-filter_context
根據相關資料，我們找到了答案：

「虎山雄風飛揚」是國立清華大學的校歌。

以下是一些有關資訊：
- 國家：中囯
  - 地區: 台北、新竹附近

這首學校之所以被稱為 "核心問題" 的原因，是因爲它與台湾教育相關。
     2-filter_context
根據提供的資訊，我們可以確定以下內容：

**答案**: "虎山雄風飛揚" 是光華國小（南投縣）的校歌。

相關數字:

* 時間: 2008年9月16日03：13；34
 

其他有用的信息：
 
- 校名 : 光 華 國 小 ( 南 投 縣 )
 - 地區/地區名稱(不明確)
     3-filter_context
根據提供的資料，我們可以找到相關資訊：

「虎山雄風飛揚」是哪間學校校歌？

答案：無法確定，但有一個可能與台北或新竹附近的地方有關。

在PDF文件中，沒有明顯提到任何一所特定的学校名稱。然而，在第320頁的內容（Content 3.20）似乎包含了一些相關資訊，但是由於資料過長，我們無法直接擷取出來。但是，可以看到有一個可能與學校有關的地方。

如果你能提供更多上下文或線索，或者更明確

In [5]:
# Combine the results into one file.
STUDENT_ID = "r12942009"
with open(f'./{STUDENT_ID}.txt', 'w') as output_f:
    for id in range(1,91):
        with open(f'./{STUDENT_ID}_{id}.txt', 'r') as input_f:
            answer = input_f.readline().strip()
            print(answer, file=output_f)